In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from wordcloud import WordCloud

from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
training = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/train.csv')
test = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/test.csv')
test_labels = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/test_labels.csv')

In [ ]:
#training = pd.read_csv("C:/Users/Falak/Desktop/NLP/train.csv")
#test = pd.read_csv("C:/Users/Falak/Desktop/NLP/test.csv")
#test_labels = pd.read_csv("C:/Users/Falak/Desktop/NLP/test_labels.csv")

In [2]:
training = pd.read_csv('C:/Users/Alvis/STA142A/train.csv')
test = pd.read_csv('C:/Users/Alvis/STA142A/test.csv')
test_labels = pd.read_csv("C:/Users/Alvis/STA142A/test_labels.csv")

In [4]:
# Preprocessing
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def preprocess(training, test):
    training['clean'] = training.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1) #removes new line character
    test['clean'] = test.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1)

    #removes urls
    training['clean'] = training.apply(lambda row: re.sub('http://\S+|https://\S+', '', row['clean']), axis=1)
    test['clean'] = test.apply(lambda row: re.sub('http://\S+|https://\S+', '',row['clean']), axis=1)

    #remove all non-alphanumeric values
    training['clean'] = training.apply(lambda row: re.sub('[^A-Za-z0-9\' ]+', '',row['clean']), axis=1)
    test['clean'] = test.apply(lambda row: re.sub('[^A-Za-z0-9\' ]+', '',row['clean']), axis=1)

    #remove stopwords as they occupy major chunk of the vocabulary
    training['clean'] = training['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word in (stop)]))
    test['clean'] = test['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word in (stop)]))

    #removes all additional spaces
    training['clean'] = training.apply(lambda row: re.sub(' +', ' ', row['clean']).strip(), axis=1)
    test['clean'] = test.apply(lambda row: re.sub(' +', ' ', row['clean']).strip(), axis=1)
    
    #lowercase end result
    training['clean'] = training['clean'].str.lower()
    test = test['clean'].str.lower()

preprocess(training, test)
training.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation why edits made username hardcore m...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww he matches background colour i'm seeming...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man i'm really trying edit war it's guy co...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,more i can't make real suggestions improvement...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,you sir hero any chance remember page that's
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,congratulations well use tools well talk
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,cocksucker before you piss around on my work
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,your vandalism matt shirvington article revert...
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,sorry word 'nonsense' offensive anyway i'm int...
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,alignment subject contrary dulithgow


In [ ]:
# Without k-fold cross validation
# for loop to check the accuracy score for each label

for i in range(2,8):
    X_train, X_validation = train_test_split(training, random_state = 999) #default is 25% split for validation
    Y_train = X_train.iloc[:,i]

    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
    tfidf_vectorizer.fit_transform(X_train['clean'].values)

    train_feature_set=tfidf_vectorizer.transform(X_train['clean'].values)
    test_feature_set=tfidf_vectorizer.transform(X_validation['clean'].values)

    scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
    model = scikit_log_reg.fit(train_feature_set, Y_train) #Model build using training data cleaned comments

    Y_predicted = model.predict(test_feature_set) #Y_predicted are the predicted labels from our model
    Y_predicted_prob = model.predict_proba(test_feature_set) #proba model shows two columns: 1st column is prob that it is classified as 0, 2nd column is prob that it is classified as 1
    
    #Predicted Accuracy score

    Y_validation = X_validation.iloc[:,i]
    print(f"The classification accuracy of the fitted model for the {training.columns[i]} label is {accuracy_score(Y_validation, Y_predicted)}")

In [5]:
#k fold crossvalidation
from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

kf2 = KFold(n_splits = 2)
kf = KFold(n_splits = 5)
kf10 = KFold(n_splits = 10)

#comments -> numbers
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
tfidf_vectorizer.fit_transform(training['clean'].values)

#cleaned complete training set
training_feature_set = tfidf_vectorizer.transform(training['clean'].values)

scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
training_feature_set

<159571x235885 sparse matrix of type '<class 'numpy.float64'>'
	with 4679912 stored elements in Compressed Sparse Row format>

In [6]:
#NEW CHUNK - kNN implementation with sample of data
import sys, os
from sklearn.neighbors import KNeighborsClassifier

#comments -> numbers
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
tfidf_vectorizer.fit_transform(training['clean'].values)

#cleaned complete training set - sampled for 20000 observations since KNN is slow
training_feature_set = tfidf_vectorizer.transform(np.random.choice(training['clean'].values, 20000))

knn = KNeighborsClassifier(n_neighbors = 5) #5-neighbors becasue KNN is slow

def kNNFit_NoMessage(X, Y, knn_obj):
    old_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    
    model = knn_obj.fit(X, Y) #Model build using training data cleaned comments
    
    sys.stdout = old_stdout
    
    return(model)

def calckNNMSE(FeatureSet, Class):
    MSE = []
    
    for train_index, valid_index in kf.split(training_feature_set): #5-folds because KNN is slow
        X_train, X_valid = training_feature_set[train_index], training_feature_set[valid_index]
        Y_train, Y_valid = training[Class][train_index], training[Class][valid_index] #labels

        #Model build using training data cleaned comments
        model = kNNFit_NoMessage(X_train, Y_train, knn) 

        # predicted y labels using X validation set
        Y_predicted = model.predict(X_valid)
        #Y_predicted = model.predict_proba(X_valid)
        score = accuracy_score(Y_valid, Y_predicted)

        MSE.append(score)

    return(np.mean(MSE))

classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

mse_list = {"toxic":[],"severe_toxic":[],"obscene":[],"threat":[],"insult":[],"identity_hate":[]}

for c in classes:
    mse_list[c].append(calckNNMSE(training_feature_set, c))
    
mse_list

{'toxic': [0.89125],
 'severe_toxic': [0.9889000000000001],
 'obscene': [0.9460000000000001],
 'threat': [0.99665],
 'insult': [0.9489000000000001],
 'identity_hate': [0.9909500000000001]}

In [57]:
#NEW CHUNK - svm implementation
import sys, os
from sklearn.svm import SVC

#comments -> numbers
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
tfidf_vectorizer.fit_transform(training['clean'].values)

#cleaned complete training set
training_feature_set = tfidf_vectorizer.transform(np.random.choice(training['clean'].values, 10000))

svm = SVC(kernel = "linear", C = 0.25, random_state = 999)

def svmFit_NoMessage(X, Y, svm_obj):
    old_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    
    model = svm_obj.fit(X, Y) #Model build using training data cleaned comments
    
    sys.stdout = old_stdout
    
    return(model)

def calcsvmMSE(FeatureSet, Class):
    MSE = []
    
    for train_index, valid_index in kf.split(training_feature_set): #kf = 5 folds
        X_train, X_valid = training_feature_set[train_index], training_feature_set[valid_index]
        Y_train, Y_valid = training[Class][train_index], training[Class][valid_index] #labels

        #Model build using training data cleaned comments
        model = svmFit_NoMessage(X_train, Y_train, svm) 

        # predicted y labels using X validation set
        Y_predicted = model.predict(X_valid)
        #Y_predicted = model.predict_proba(X_valid)
        score = accuracy_score(Y_valid, Y_predicted)

        MSE.append(score)

    return(np.mean(MSE))

classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

mse_list = {"toxic":[],"severe_toxic":[],"obscene":[],"threat":[],"insult":[],"identity_hate":[]}

for c in classes:
    mse_list[c].append(calcsvmMSE(training_feature_set, c))
    
mse_list

{'toxic': [0.9029],
 'severe_toxic': [0.9899000000000001],
 'obscene': [0.9473],
 'threat': [0.9967],
 'insult': [0.9505999999999999],
 'identity_hate': [0.9916]}

In [7]:
# takes in feature set and class, returns accuracy score of logistic regression model for that class
import sys, os
def LogRegFit_NoMessage(X, Y, logreg_obj):
    old_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    
    model = logreg_obj.fit(X, Y) #Model build using training data cleaned comments
    
    sys.stdout = old_stdout
    
    return(model)

def calcClassMSE(FeatureSet, Class):
    MSE = []
    
    for train_index, valid_index in kf10.split(training_feature_set):
        X_train, X_valid = training_feature_set[train_index], training_feature_set[valid_index]
        Y_train, Y_valid = training[Class][train_index], training[Class][valid_index] #labels

        #Model build using training data cleaned comments
        model = LogRegFit_NoMessage(X_train, Y_train, scikit_log_reg) 

        # predicted y labels using X validation set
        Y_predicted = model.predict(X_valid)
        #Y_predicted = model.predict_proba(X_valid)
        score = accuracy_score(Y_valid, Y_predicted)

        MSE.append(score)

    return(np.mean(MSE))

classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

mse_list = {"toxic":[],"severe_toxic":[],"obscene":[],"threat":[],"insult":[],"identity_hate":[]}

for c in classes:
    mse_list[c].append(calcClassMSE(training_feature_set, c))
    
mse_list

{'toxic': [0.90105],
 'severe_toxic': [0.9889000000000001],
 'obscene': [0.9461],
 'threat': [0.99665],
 'insult': [0.9488],
 'identity_hate': [0.99095]}

In [25]:
#applied cross validation on one class 'toxic'
MSE = []
for train_index, valid_index in kf10.split(training_feature_set):
    X_train, X_valid = training_feature_set[train_index], training_feature_set[valid_index]
    Y_train, Y_valid = training['toxic'][train_index], training['toxic'][valid_index]
    
    #Model build using training data cleaned comments
    model = LogRegFit_NoMessage(X_train, Y_train, scikit_log_reg)
    
    # predicted y labels using X validation set
    Y_predicted = model.predict(X_valid)
    #Y_predicted = model.predict_proba(X_valid)
    score = accuracy_score(Y_valid, Y_predicted)
    
    MSE.append(score)
    
np.mean(MSE)

'threat'

In [ ]:
classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

mse_list = {"toxic":[],"severe_toxic":[],"obscene":[],"threat":[],"insult":[],"identity_hate":[]}

for c in classes:
    mse_list[c].append(calcClassProba(training_feature_set, c))